# 레시피 url 및 조리 시간 크롤링

In [260]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

In [261]:
# 드라이버 초기화
driver = webdriver.Chrome()

In [138]:
# url 설정
# difficulty = 10 : 초보, 20 : 보통
# 페이지: 60, 페이지: 46
beginner_url = "https://www.menupan.com/Cook/RecipeRe.asp?difficulty=10&page="
intermediate_url = "https://www.menupan.com/Cook/RecipeRe.asp?difficulty=20&page="

In [139]:
# 레시피 url 저장할 리스트
recipe_urls = []
# 요리시간 저장할 리스트
coocking_times = []

for page_num in range(1, 61):
    url = f"{beginner_url}{page_num}"
    driver.get(url)
    time.sleep(1)

    # 레시피 url 저장
    # 해당 페이지의 레시피 ID가져오기
    recipe_IDs = driver.find_elements(By.CSS_SELECTOR, "span.link a[href*='goRecipeView']")

    # 모든 요소를 순회하면서 데이터 추출
    for ID in recipe_IDs:
        recipe_ID = ID.get_attribute('href').split('(')[1].split(')')[0]
        recipe_urls.append(f"https://www.menupan.com/Cook/RecipeView.asp?cookid={recipe_ID}")

    # 요리시간 저장
    time_elements = driver.find_elements(By.XPATH, "//td[contains(text(), '조리시간')]")
    for element in time_elements:

        time_info = element.text.split('조리시간:')[1].split(' ')[0].strip()
        if (time_info == ''):  # '조리시간'이 없는 경우
            coocking_times.append('N/A')
        elif (time_info[-1] == '분'):
            coocking_times.append(time_info.split('분')[0])
        elif (time_info[-1].isdigit() == True):
            coocking_times.append(time_info)
        else:
            coocking_times.append(str(int(time_info.split('시')[0])*60))




# url이랑 조리시간 함께 저장
recipe_info = pd.DataFrame({
    'url': recipe_urls, 
    'cooking_time': coocking_times
})

# CSV 파일로 저장
recipe_info.to_csv('recipe_info_beginner.csv', index=False, encoding='utf-8-sig')


KeyboardInterrupt: 

In [ ]:
# 레시피 url 저장할 리스트
recipe_urls = []
# 요리시간 저장할 리스트
coocking_times = []
for page_num in range(1, 47):
    url = f"{intermediate_url}{page_num}"
    driver.get(url)
    time.sleep(1)

    # 레시피 url 저장
    # 해당 페이지의 레시피 ID가져오기
    recipe_IDs = driver.find_elements(By.CSS_SELECTOR, "span.link a[href*='goRecipeView']")

    # 모든 요소를 순회하면서 데이터 추출
    for ID in recipe_IDs:
        recipe_ID = ID.get_attribute('href').split('(')[1].split(')')[0]
        recipe_urls.append(f"https://www.menupan.com/Cook/RecipeView.asp?cookid={recipe_ID}")

    # 요리시간 저장
    time_elements = driver.find_elements(By.XPATH, "//td[contains(text(), '조리시간')]")

    for element in time_elements:

        time_info = element.text.split('조리시간:')[1].split(' ')[0].strip()
        if (time_info == ''):  # '조리시간'이 없는 경우
            coocking_times.append('N/A')
        elif (time_info[-1] == '분'):
            coocking_times.append(time_info.split('분')[0])
        elif (time_info[-1].isdigit() == True):
            coocking_times.append(time_info)
        else:
            coocking_times.append(str(int(time_info.split('시')[0])*60))


# url이랑 조리시간 함께 저장
recipe_info = pd.DataFrame({
    'url': recipe_urls, 
    'cooking_time': coocking_times
})

# CSV 파일로 저장
recipe_info.to_csv('recipe_info_intermediate.csv', index=False, encoding='utf-8-sig')

# 위에 레시피 url을 통해서 레시피 정보 크롤링

In [265]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import os

In [266]:
beginner_df = pd.read_csv('recipe_info_beginner.csv')
intermediate_df = pd.read_csv('recipe_info_intermediate.csv')
base_url = "https://www.menupan.com"
beginner_directory = 'beginner_img'
intermediate_directory = 'intermediate_img'

if not os.path.exists(beginner_directory):
    os.makedirs(beginner_directory)

if not os.path.exists(intermediate_directory):
    os.makedirs(intermediate_directory)

In [267]:
recipe_urls = beginner_df['url'].tolist()
data = []
cooking_time_counter = 0


for url in recipe_urls:
    driver.get(url)
    time.sleep(1)
    # 메뉴 이름 가져오기 (string)
    menu_name = driver.find_element(By.CLASS_NAME, 'wrap_top').find_element(By.TAG_NAME, 'h2').text
    # 방법 분류 가져오기 (list)
    method_category = driver.find_element(By.CLASS_NAME, 'name').find_element(By.TAG_NAME, 'a').text
    method_list = [method.strip() for method in method_category.split(',')]
    # 국가 분류 가져오기 (string)
    country = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div[2]/dl[2]/dd/a').text
    # 테마 분류 가져오기 (list)
    theme_category = driver.find_element(By.CLASS_NAME, 'Theme').text
    theme_list = [theme.strip() for theme in theme_category.split(',')]
    # 칼로리 가져오기 (float)
    calorie = float(driver.find_element(By.XPATH, "/html/body/div/div[2]/div[2]/div[2]/dl[6]/dd/a").text.split('k')[0])
    # 조회수 가져오기 (int)
    view_text = driver.find_element(By.CLASS_NAME, 'visitor').find_element(By.TAG_NAME, 'strong').text
    view = int(view_text.replace('명', '').replace(',', ''))
    
    # 분량, 주재료 부재료 가져오기 (int, string, string)
    infoTable = driver.find_element(By.CSS_SELECTOR, "div.infoTable ul.tableLR")
    details = infoTable.find_elements(By.TAG_NAME, "li")
    amount_text = details[0].find_element(By.TAG_NAME, "dd").text
    amount = int(amount_text.split('인분')[0])
    main_ingredients = [ingredient.strip() for ingredient in details[1].find_element(By.TAG_NAME, "dd").text.split(',')]
    
    if len(details) > 2:  # 부재료가 있다고 가정
        sub_ingredients = [ingredient.strip() for ingredient in details[2].find_element(By.TAG_NAME, "dd").text.split(',')]
    else:
        sub_ingredients = "N/A"
    
    # 양념 섹션 체크 및 추출
    sauce = "N/A"
    if len(details) > 3:  # 양념 섹션이 있다고 가정
        sauce = [ingredient.strip() for ingredient in details[3].find_element(By.TAG_NAME, "dd").text.split(',')]

    # 레시피 가져오기
    recipe_steps = []
    steps_elements = driver.find_elements(By.CSS_SELECTOR, ".recipePlus dt")
    for step_element in steps_elements:
        step_text = step_element.text.strip()
        recipe_steps.append(step_text)

    menu = {
        '메뉴 이름': menu_name,
        '방법 분류': method_list,
        '국가 분류': country,
        '테마 분류': theme_list,
        '난이도 분류': '초보',
        '칼로리': calorie,
        '조회수': view,
        '분량': amount,
        '주재료 이름': main_ingredients,
        #'주재료 분량': ingredient_amounts,
        '부재료 이름': sub_ingredients,
        #'부재료 분량': sub_ingredient_amounts,
        '양념': sauce,
        '레시피': recipe_steps,
        '조리시간': beginner_df['cooking_time'][cooking_time_counter]
    }
    cooking_time_counter += 1
    data.append(menu)

    # 이미지 다운로드
    # Find image by tag and class if there are multiple img tags
    image_element = driver.find_element(By.CSS_SELECTOR, "img.img")
    img_url = image_element.get_attribute('src')
    response = requests.get(img_url)
    if response.status_code == 200:
        with open(f"{beginner_directory}/{menu_name}.jpg", 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download image for {menu_name}")


df = pd.DataFrame(data)
df.to_csv('beginner_recipe.csv', index=False, encoding='utf-8-sig')
    


In [268]:
recipe_urls = intermediate_df['url'].tolist()
data = []
cooking_time_counter = 0

for url in recipe_urls:
    driver.get(url)
    time.sleep(1)
    # 메뉴 이름 가져오기 (string)
    menu_name = driver.find_element(By.CLASS_NAME, 'wrap_top').find_element(By.TAG_NAME, 'h2').text
    # 방법 분류 가져오기 (list)
    method_category = driver.find_element(By.CLASS_NAME, 'name').find_element(By.TAG_NAME, 'a').text
    method_list = [method.strip() for method in method_category.split(',')]
    # 국가 분류 가져오기 (string)
    country = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div[2]/dl[2]/dd/a').text
    # 테마 분류 가져오기 (list)
    theme_category = driver.find_element(By.CLASS_NAME, 'Theme').text
    theme_list = [theme.strip() for theme in theme_category.split(',')]
    # 칼로리 가져오기 (int)
    calorie = float(driver.find_element(By.XPATH, "/html/body/div/div[2]/div[2]/div[2]/dl[6]/dd/a").text.split('k')[0])
    # 조회수 가져오기 (int)
    view_text = driver.find_element(By.CLASS_NAME, 'visitor').find_element(By.TAG_NAME, 'strong').text
    view = int(view_text.replace('명', '').replace(',', ''))
    
    # 분량, 주재료 부재료 가져오기 (int, string, string)
    infoTable = driver.find_element(By.CSS_SELECTOR, "div.infoTable ul.tableLR")
    details = infoTable.find_elements(By.TAG_NAME, "li")
    amount_text = details[0].find_element(By.TAG_NAME, "dd").text
    amount = int(amount_text.split('인분')[0])
    main_ingredients = [ingredient.strip() for ingredient in details[1].find_element(By.TAG_NAME, "dd").text.split(',')]

    if len(details) > 2:  # 부재료가 있다고 가정
        sub_ingredients = [ingredient.strip() for ingredient in details[2].find_element(By.TAG_NAME, "dd").text.split(',')]
    else:
        sub_ingredients = "N/A"

    # 양념 섹션 체크 및 추출
    sauce = "N/A"
    if len(details) > 3:  # 양념 섹션이 있다고 가정
        sauce = [ingredient.strip() for ingredient in details[3].find_element(By.TAG_NAME, "dd").text.split(',')]

    # 레시피 가져오기
    recipe_steps = []
    steps_elements = driver.find_elements(By.CSS_SELECTOR, ".recipePlus dt")
    for step_element in steps_elements:
        step_text = step_element.text.strip()
        recipe_steps.append(step_text)

    menu = {
        '메뉴 이름': menu_name,
        '방법 분류': method_list,
        '국가 분류': country,
        '테마 분류': theme_list,
        '난이도 분류': '보통',
        '칼로리': calorie,
        '조회수': view,
        '분량': amount,
        '주재료 이름': main_ingredients,
        #'주재료 분량': ingredient_amounts,
        '부재료 이름': sub_ingredients,
        #'부재료 분량': sub_ingredient_amounts,
        '양념': sauce,
        '레시피': recipe_steps,
        '조리시간': beginner_df['cooking_time']
    }
    cooking_time_counter += 1
    data.append(menu)

    # 이미지 다운로드
    # Find image by tag and class if there are multiple img tags
    image_element = driver.find_element(By.CSS_SELECTOR, "img.img")
    img_url = image_element.get_attribute('src')
    response = requests.get(img_url)
    if response.status_code == 200:
        with open(f"{intermediate_directory}/{menu_name}.jpg", 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download image for {menu_name}")
    

df = pd.DataFrame(data)
df.to_csv('intermediate_recipe.csv', index=False, encoding='utf-8-sig')

# 이미지 누락 확인 (beginner_recipe.csv에서 메뉴 이름이 중복)

In [278]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = 'beginner_recipe.csv'
df = pd.read_csv(csv_file_path)

# Find duplicates based on '메뉴 이름' or another suitable identifier
duplicates = df[df.duplicated('메뉴 이름', keep=False)]

# Print the duplicates to review them
print("Duplicate entries based on 메뉴 이름:")
print(duplicates)

# Alternatively, you can count the number of duplicates
duplicate_count = df.duplicated('메뉴 이름').sum()
print(f"There are {duplicate_count} duplicate entries based on 메뉴 이름.")


Duplicate entries based on 메뉴 이름:
       메뉴 이름                방법 분류 국가 분류  \
37       참치죽     ['밥', '죽', '스프']    한식   
38       참치죽     ['밥', '죽', '스프']    한식   
99     알감자조림               ['조림']    한식   
100    알감자조림               ['조림']    한식   
120     두부김치               ['구이']    한식   
213  해파리해물냉채  ['나물', '생채', '샐러드']    한식   
214  해파리해물냉채  ['나물', '생채', '샐러드']    한식   
266       화전          ['떡', '한과']    한식   
267       화전          ['떡', '한과']    한식   
332      대구전               ['부침']    한식   
333      대구전               ['부침']    한식   
369     두부김치               ['볶음']    한식   
449    과일샐러드  ['나물', '생채', '샐러드']    서양   
450    과일샐러드  ['나물', '생채', '샐러드']    서양   
512    토마토소스              ['양념장']    서양   
513    토마토소스              ['양념장']    서양   
527     모양쿠키          ['빵', '과자']    서양   
528     모양쿠키          ['빵', '과자']    서양   
556    딸기스무디               ['음료']    서양   
557    딸기스무디               ['음료']    서양   
599     유부초밥     ['밥', '죽', '스프']    일본   
600     유부초밥     ['밥